Code pour projet crypto

Import des librairies

**Date limite :** Mercredi 27 mars 2024

**Groupe TD DIA 6 :**

Henri Serano, Sara Thibierge, Eloi Seidlitz

In [14]:
import hashlib
import random 
import gmpy2 as gm
import math
import time
import base64
import os, binascii
from backports.pbkdf2 import pbkdf2_hmac
import pyaes, pbkdf2, secrets
import nbdime


In [15]:
!nbdime config-git --enable

## Exercice 1

1. Trouver une chaîne de caractères (contenant vos noms et prénoms) dont le hash SHA256 se termine par le plus de zéros possible (en hexadécimal).

2. Mesurer le temps moyen pour obtenir n et n+1 zéros en fin de chaîne (n = 5) et calculer le rapport Tn+1/Tn.

### Partie 1

Nous allons faire une fonction permettant de trouver une combinaison du chiffrage de la phrase avec l'algorithme sha256 qui finit avec un nombre finit de 0

In [2]:
def trouver_chaine(base, zeros_cibles):
        compteur = 0
        while True:
            chaine_test = base + str(compteur)
            hash_resultat = sha256_hash(chaine_test)
            if hash_resultat.endswith('0' * zeros_cibles):
                return chaine_test, compteur, hash_resultat
            compteur += 1

def sha256_hash(chaine):
        return hashlib.sha256(chaine.encode()).hexdigest()

Cette fonction permet d'executer la fonction et de trouver les différentes statistique en lien avec ce cryptage

In [3]:
def trouver_chaine_et_stats(base, zeros_cibles, iterations_stat=5):
    

    # Première partie : Trouver la chaîne
    chaine_trouvee, compteur_trouve, hash_trouve = trouver_chaine(base, zeros_cibles)

    # Deuxième partie : Statistiques de temps
    temps_n = []
    temps_n_plus_1 = []
    
    for _ in range(iterations_stat):
        debut = time.time()
        trouver_chaine(base, zeros_cibles)
        fin = time.time()
        temps_n.append(fin - debut)

        debut = time.time()
        trouver_chaine(base, zeros_cibles + 1)
        fin = time.time()
        temps_n_plus_1.append(fin - debut)

    temps_moyen_n = sum(temps_n) / len(temps_n)
    temps_moyen_n_plus_1 = sum(temps_n_plus_1) / len(temps_n_plus_1)

    rapport_temps = temps_moyen_n_plus_1 / temps_moyen_n

    return {
        "chaine_trouvee": chaine_trouvee,
        "hash_trouve": hash_trouve,
        "compteur_trouve": compteur_trouve,
        "temps_moyen_n": temps_moyen_n,
        "temps_moyen_n_plus_1": temps_moyen_n_plus_1,
        "rapport_temps": rapport_temps
    }


base_chaine = "Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA6"
zeros_cibles = 4 # Nombre de zéros à la fin du hash
resultats = trouver_chaine_et_stats(base_chaine, zeros_cibles)
resultats


{'chaine_trouvee': 'Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA677031',
 'hash_trouve': '31e6f8e6377ad1c9895ef7c22b0ef9a252c0f0ec9d062178c58b416bcb470000',
 'compteur_trouve': 77031,
 'temps_moyen_n': 0.24442505836486816,
 'temps_moyen_n_plus_1': 0.7190749168395996,
 'rapport_temps': 2.9419034269644837}

## Exercice 2

1. Chiffrer un texte d'environ une demi-page (format .txt) avec AES256-CTR-PBKDF2 en utilisant un mot de passe et une valeur de compteur (IV) de 6 à 9 chiffres (sans 0).

2. Constituez un entier N en accolant le mot de passe et l'IV séparés par quatre 0.

3. Chiffrer N avec ElGamal en utilisant les nombres p, g et A fournis.

4. Sauvegarder le fichier chiffré avec l'extension .enc.

Nous allons importer le texte directement depuis un fichier txt

In [4]:
with open("format.txt", "r",encoding="utf-8") as file:
    text = str(file.read())
text

"Le marché des sneakers en France a connu une croissance exponentielle au cours des dernières années, devenant un phénomène de mode et un investissement lucratif pour de nombreux amateurs et professionnels. \nCe texte explore les multiples faceteurs qui sous-tendent le commerce de sneakers dans l'Hexagone, en mettant en lumière ses dynamiques économiques, culturelles et sociales.\nD'un point de vue économique, le commerce de sneakers en France est stimulé par une demande croissante pour des modèles rares et exclusifs. \nLes éditions limitées et les collaborations entre marques de renom et designers célèbres créent une urgence d'achat, poussant les prix sur le marché secondaire à atteindre des sommets vertigineux. \nCette situation a engendré l'émergence d'une véritable économie parallèle, où les sneakers sont non seulement des objets de mode, mais également des actifs spéculatifs.\n\nSur le plan culturel, les sneakers occupent une place prépondérante dans l'univers de la mode et de la 

Nous allons initialisé nos différentes clés de sorte que l'on ait le bon nombre de 0

In [5]:
def generate_non_zero_number(length,password):
    return int(''.join(secrets.choice(password) for _ in range(length)))
password = "123456789"
salt = os.urandom(16)
key = pbkdf2.PBKDF2(password, salt).read(32)
iv = generate_non_zero_number(random.randint(6, 9),password)

N = int(password + "0000" + str(iv)) 
print("password: ",password)
print("IV : ",iv)
print("N : ",N)



password:  123456789
IV :  96662139
N :  123456789000096662139


Après avoir généré les différentes clés nous allons crypté avec l'algo AES avec le mode CTR

In [6]:
#On est obliger de recrée un mot de pass unique pour chiffrer le texte
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
ciphertext = aes.encrypt(text)
with open("project.enc", "wb") as encrypted_file:
    encrypted_file.write(ciphertext)
print(ciphertext)

b'h\x87\n\x9c\x98s\xd8\xb0\xad2\x14\xf8aSOs\x99\x99\xca\\\xa0V2\x96\n\x063i\xea)QJV\x95\xce9\x0b\xe6Ci\xd2\x066\x7f\xf4\xe86\xad\x11\r;Qq\xf6\xcf\x1dC\x96dp\x1b*\xf9\x9a#\xe9\x18\x1c8u\xf0\xaa\x8b:H\x96\xa6\'\xf7*\xd2\xe7\x83"dL\x1b\xc1`5g\xad\x15\xb3\x18a,\x9b\xcb_\xdd\x0c?\xca,\x84R+\xd1\xa42\xe7m\xfd\xf8\xaa\xe4\xc5\xca\xc0[\xcfY\xbf\xf2)\xa1\x07\xdb<\x18>\x0b\xbb\xe6jz\x01\xea\xc8\xb4\x08R\xff\xdea\x05\xbd\xc7gX\xfd\xcb\xadt\xcd\x0f\x10\xe9\xf1\xc0I\x00\xb9\x86\xe4\x94\x81\xaa\x8f\xda\x7f\xbc\xfb7\xba\x90\xe0\xe8^\xac\x19\x86Tkv\xc8\x1d\xa57\xa5P\xc6,\xda\x0c\xf2\xe6\xb1\x12\xc33Yo\x80\xcc$f\x87}\xde\xd4\x17\x90UUD\x982\xb8Qtu q\x11\x85\xb8T&g\x98\xbcK\x89\x1dE#\x98gW?w\xc7xGP;A1\xf9VT\xf8t/\xe8\x1a\x8dO\xaf.\xeex\x16\x95\xb9\x0b\x93%.\x02\xbd\x02\x14\x7fT~\xfd9\x01T\x8dc\x88qL\xc25Gu\xf5\x1f\xe9G\xa3\x06F~\x16_I3\xfc\x82\x12\xea\xdd\xdfP\xe0u\xbf^\xf8\x98qT\xd2c\xed\xe1\x83\xbc\xe5\x8b\x9f)^\x154+\xdfsx\xb9:N\x86\\\x1d\x1dq\x0bm\xbc>\xac\xe6:\x82n\xd5G?|\x9c\xc9\x83\x122~\x9ch\x8f

Petites fonction permettant de tester si le décriptage de texte fonctionne correctement

In [7]:
#Et du coup de le regénéré
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
print(aes.decrypt(ciphertext))

b"Le march\xe9 des sneakers en France a connu une croissance exponentielle au cours des derni\xe8res ann\xe9es, devenant un ph\xe9nom\xe8ne de mode et un investissement lucratif pour de nombreux amateurs et professionnels. \nCe texte explore les multiples faceteurs qui sous-tendent le commerce de sneakers dans l'Hexagone, en mettant en lumi\xe8re ses dynamiques \xe9conomiques, culturelles et sociales.\nD'un point de vue \xe9conomique, le commerce de sneakers en France est stimul\xe9 par une demande croissante pour des mod\xe8les rares et exclusifs. \nLes \xe9ditions limit\xe9es et les collaborations entre marques de renom et designers c\xe9l\xe8bres cr\xe9ent une urgence d'achat, poussant les prix sur le march\xe9 secondaire \xe0 atteindre des sommets vertigineux. \nCette situation a engendr\xe9 l'\xe9mergence d'une v\xe9ritable \xe9conomie parall\xe8le, o\xf9 les sneakers sont non seulement des objets de mode, mais \xe9galement des actifs sp\xe9culatifs.\n\nSur le plan culturel, les s

Pour le cryptage ElGalmal nous allons prendre les variables que l'énoncé nous donnes et généré une variable B permettant de retrouver notre variable N qui contient la clé et le compteur iv

In [8]:
p = 7946851324679854613245823
g = 5
A = 7579501795988122393422986

b = random.SystemRandom().randint(2, p-2)

# Calcul de B et C pour ElGamal
B = pow(g, b, p)
K = pow(A, b, p)  # Clé secrète partagée
C = (K * N) % p  # Chiffrement de N avec la clé secrète partagée

B, C

(6562940414057677943035811, 5225592046137241832847437)

In [9]:
"""K = pow(B, g, p)

# Calculer l'inverse modulaire de K
K_inv = gm.invert(K, p)

# Récupérer N
N = (C * K_inv) % p

print(N)

password, iv = str(N)[:-13], int(str(N)[-10:])  # Ceci est un exemple; ajustez selon la structure réelle de N
print(password,iv)
# Dérivation de la clé AES à partir du mot de passe
key = pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 100000, dklen=32)

aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))

# Lire le fichier chiffré
with open("project.enc", "rb") as encrypted_file:
    ciphertext = encrypted_file.read()
# Déchiffrer le contenu
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
print(aes.decrypt(ciphertext)) """

'K = pow(B, g, p)\n\n# Calculer l\'inverse modulaire de K\nK_inv = gm.invert(K, p)\n\n# Récupérer N\nN = (C * K_inv) % p\n\nprint(N)\n\npassword, iv = str(N)[:-13], int(str(N)[-10:])  # Ceci est un exemple; ajustez selon la structure réelle de N\nprint(password,iv)\n# Dérivation de la clé AES à partir du mot de passe\nkey = pbkdf2_hmac(\'sha256\', password.encode(\'utf-8\'), salt, 100000, dklen=32)\n\naes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))\n\n# Lire le fichier chiffré\nwith open("project.enc", "rb") as encrypted_file:\n    ciphertext = encrypted_file.read()\n# Déchiffrer le contenu\naes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))\nprint(aes.decrypt(ciphertext)) '

## Exercice 3

* Démonstration de la signature ElGamal ou RSA (avec des nombres différents de ceux du cours).
* Exemple d'utilisation du RSA, Diffie-Hellman, ElGamal ou de la signature ElGamal dans Python ou openssl (contexte réel).

#### 1. Démonstration de la signature ElGamal ou RSA (avec des nombres différents de ceux du cours).

In [10]:
# Fonction pour calculer l'inverse modulo
def inverse_modulo(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

# Fonction pour calculer la signature
def sign_elgamal(m, x, p, g):
    # Sélection d'un nombre aléatoire k
    k = random.randint(2, p - 2)
    while pow(k, -1, p - 1) == 0:  # Assure que k est premier avec p-1
        k = random.randint(2, p - 2)
    r = pow(g, k, p)
    k_inv = inverse_modulo(k, p - 1)
    s = (m - x * r) * k_inv % (p - 1)
    return r, s

# Fonction pour vérifier la signature
def verify_elgamal(m, r, s, y, p, g):
    v1 = pow(g, m, p)
    v2 = (pow(y, r, p) * pow(r, s, p)) % p
    return v1 == v2

# Paramètres ElGamal
p = 257
g = 3
x = 97

# Clé publique
y = pow(g, x, p)

# Message à signer
m = 123

# Signature
r, s = sign_elgamal(m, x, p, g)

# Vérification de la signature
valid = verify_elgamal(m, r, s, y, p, g)

print("Signature valide :", valid)


Signature valide : True


#### 2. Exemple d'utilisation du RSA, Diffie-Hellman, ElGamal ou de la signature ElGamal dans Python ou openssl (contexte réel).

Supposons que nous ayons un fichier texte appelé "format.txt" contenant le message à signer.



**a) Génération des clés RSA :**
Tout d'abord, nous devons générer une paire de clés RSA privée/publique à l'aide d'OpenSSL.

In [11]:
!openssl genpkey -algorithm RSA -out private_key.pem
!openssl rsa -pubout -in private_key.pem -out public_key.pem

......+..+..........+...+..+......+...............+....+...+..+......+....+...+...........+.+++++++++++++++++++++++++++++++++++++++*.+...+........+....+.....+++++++++++++++++++++++++++++++++++++++*...+.........+..............+.+......+...+..+.........+....+......+.....+................+..+.+....................+......................+........+............+...+.........+.+..............+...+.......+...............+...++++++
....+..+....+++++++++++++++++++++++++++++++++++++++*......+....+..+...+...+....+...+++++++++++++++++++++++++++++++++++++++*.......+.........+..+...............+....+.........+..+...+..........+..+...+...+....+......+...+..+...+.+......+........+.+.....+.+.....+.......++++++
writing RSA key


Ces commandes génèrent une clé privée private_key.pem et une clé publique correspondante `public_key.pem`.



**b) Signature du message :**

Ensuite, nous signons le message avec notre clé privée.

In [12]:
!openssl dgst -sha256 -sign private_key.pem -out signature.bin format.txt

Cela crée une signature du message dans le fichier signature.bin.


**c) Vérification de la signature :**

Pour vérifier la signature avec la clé publique correspondante, nous utilisons :

In [13]:
!openssl dgst -sha256 -verify public_key.pem -signature signature.bin format.txt

Verified OK


Si la vérification réussit, OpenSSL affichera "Verified OK".
